In [1]:
import torch

# Define a tensor with requires_grad=True
x = torch.tensor(2.0, requires_grad=True)

# Define a scalar function of x
z = x**3  # z = x^3

# Compute the first derivative, enabling create_graph=True
grad_first = torch.autograd.grad(z, x, create_graph=True)

print(grad_first)

# grad_first is a tuple; extract the first element
print("First derivative:", grad_first[0])  # Should print 3x^2 for x=2.0, i.e., 12.0

# Compute the second derivative (d²z/dx²) using the first derivative
grad_second = torch.autograd.grad(grad_first[0], x)

# grad_second is also a tuple; extract the first element
print("Second derivative:", grad_second[0])  # Should print 6x for x=2.0, i.e., 12.0


(tensor(12., grad_fn=<MulBackward0>),)
First derivative: tensor(12., grad_fn=<MulBackward0>)
Second derivative: tensor(12.)
